# Sources

Upstox - https://upstox.com/developer/api-documentation/instruments/
NSE200 - https://www.nseindia.com/products-services/indices-nifty200-index

Vlookup - "C:\Users\perry\OneDrive\nifty200\ind_nifty200list.xlsx"

In [12]:
# open xlsx with nifty200 symbols and upstox instrument key
import pandas as pd

# Specify the file path
file_path = 'ind_nifty200list.xlsx'

# Read the Excel file
df = pd.read_excel(file_path)

# Print the DataFrame
print(df)


                    Company Name                Industry      Symbol Series  \
0                 ABB India Ltd.           Capital Goods         ABB     EQ   
1                       ACC Ltd.  Construction Materials         ACC     EQ   
2          APL Apollo Tubes Ltd.           Capital Goods   APLAPOLLO     EQ   
3     AU Small Finance Bank Ltd.      Financial Services      AUBANK     EQ   
4    Adani Energy Solutions Ltd.                   Power  ADANIENSOL     EQ   
..                           ...                     ...         ...    ...   
195                  Voltas Ltd.       Consumer Durables      VOLTAS     EQ   
196                   Wipro Ltd.  Information Technology       WIPRO     EQ   
197                Yes Bank Ltd.      Financial Services     YESBANK     EQ   
198                  Zomato Ltd.       Consumer Services      ZOMATO     EQ   
199      Zydus Lifesciences Ltd.              Healthcare   ZYDUSLIFE     EQ   

        ISIN Code       instrument_key  
0    INE11

In [27]:
from datetime import date, timedelta, datetime
import requests

def datesort(crow):
    dt = crow[0]
    return int(datetime.fromisoformat(dt).timestamp())

def get_12m_returns(instkey):
    ed = str(date.today())
    sd = str(date.today() - timedelta(weeks=52))
    url = f"https://api-v2.upstox.com/historical-candle/{instkey}/month/{ed}/{sd}"
    h = {
        "Api-Version": "2.0",
        "Accept": "application/json",
        "Authorization": "***REMOVED***",
    }

    # https://upstox.com/developer/api-documentation/get-historical-candle-data/
    resp = requests.get(url, headers=h)
    if resp.status_code != 200:
        print(f"inst: {instkey} err: {resp}")
        return None
    rjson = resp.json()
    
    if rjson["status"] != "success":
        raise Exception(rjson["status"])
    
    # print(f"{rjson}")
    candles = rjson["data"]["candles"]
    candles.sort(key=datesort, reverse=True)
    # for c in candles:
    #     print(f"date: {c[0]} close: {c[4]}")

    if candles:
        start_price = candles[-1][4]
        end_price = candles[0][4]
        return (end_price - start_price)/start_price
    return 0

# print(get_12m_returns("NSE_EQ|INE084A01016"))
# print(get_12m_returns("NSE_EQ|INE498L01015"))

In [28]:
print(get_12m_returns("NSE_EQ|INE121A08PJ0"))

0


In [29]:
# Sort C200 by previous 12 months returns decreasing

import pandas as pd

sym_returns = dict()

def apply_12m_returns(row):
    # print(row)
    sym_returns[row['Symbol']] = get_12m_returns(row['instrument_key'])

df = pd.read_excel('ind_nifty200list.xlsx')
_ = df.apply(apply_12m_returns, axis='columns')

t200 = []
for k,v in sorted(sym_returns.items(), key=lambda x: x[1], reverse=True):
    t200.append({"symbol": k, "gain": v})

print(t200)


[{'symbol': 'OFSS', 'gain': 1.8992524164317355}, {'symbol': 'RVNL', 'gain': 1.467364362434591}, {'symbol': 'DIXON', 'gain': 1.378817527696349}, {'symbol': 'COCHINSHIP', 'gain': 1.1387740029542097}, {'symbol': 'BSE', 'gain': 1.113227084458851}, {'symbol': 'POLICYBZR', 'gain': 1.1125023595293526}, {'symbol': 'TRENT', 'gain': 1.0618831732106908}, {'symbol': 'OIL', 'gain': 1.0497783151954858}, {'symbol': 'ZOMATO', 'gain': 1.010751818916734}, {'symbol': 'KALYANKJIL', 'gain': 0.9601807399039819}, {'symbol': 'IREDA', 'gain': 0.9511673151750974}, {'symbol': 'CUMMINSIND', 'gain': 0.8612235545711447}, {'symbol': 'VEDL', 'gain': 0.849727327812563}, {'symbol': 'MAZDOCK', 'gain': 0.8256390021482749}, {'symbol': 'TORNTPOWER', 'gain': 0.8141029073191625}, {'symbol': 'VOLTAS', 'gain': 0.8050799816016763}, {'symbol': 'SIEMENS', 'gain': 0.7825800680796086}, {'symbol': 'JSWENERGY', 'gain': 0.7407407407407406}, {'symbol': 'HUDCO', 'gain': 0.7349741961095674}, {'symbol': 'M&M', 'gain': 0.7201919740950619},

In [31]:
for s in t200:
    print(f"{s}")

{'symbol': 'OFSS', 'gain': 1.8992524164317355}
{'symbol': 'RVNL', 'gain': 1.467364362434591}
{'symbol': 'DIXON', 'gain': 1.378817527696349}
{'symbol': 'COCHINSHIP', 'gain': 1.1387740029542097}
{'symbol': 'BSE', 'gain': 1.113227084458851}
{'symbol': 'POLICYBZR', 'gain': 1.1125023595293526}
{'symbol': 'TRENT', 'gain': 1.0618831732106908}
{'symbol': 'OIL', 'gain': 1.0497783151954858}
{'symbol': 'ZOMATO', 'gain': 1.010751818916734}
{'symbol': 'KALYANKJIL', 'gain': 0.9601807399039819}
{'symbol': 'IREDA', 'gain': 0.9511673151750974}
{'symbol': 'CUMMINSIND', 'gain': 0.8612235545711447}
{'symbol': 'VEDL', 'gain': 0.849727327812563}
{'symbol': 'MAZDOCK', 'gain': 0.8256390021482749}
{'symbol': 'TORNTPOWER', 'gain': 0.8141029073191625}
{'symbol': 'VOLTAS', 'gain': 0.8050799816016763}
{'symbol': 'SIEMENS', 'gain': 0.7825800680796086}
{'symbol': 'JSWENERGY', 'gain': 0.7407407407407406}
{'symbol': 'HUDCO', 'gain': 0.7349741961095674}
{'symbol': 'M&M', 'gain': 0.7201919740950619}
{'symbol': 'INDHOTEL

In [32]:
# select top 40 (T40) and top 20 (T20) from it.

t40 = t200[:40]
t20 = t200[:20]

In [ ]:
# Open portfolio.csv, which contains all symbols held currently, call it HXX

import pandas as pd

df = pd.read_csv("portfolio.csv")